<span style="color: green; font-family: Segoe UI; font-size: 2.5em; font-weight: 300;"> GI Tract Image Segmentation Setup: Step 1</span>


In [ ]:
import numpy as np 
import pandas as pd
from PIL import Image
import os , glob,cv2

In [ ]:
train = '../input/uw-madison-gi-tract-image-segmentation/train'
train_csv = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/train.csv')
#Exploring csv file
train_csv.info()


<p style = "color:purple; background-color:pink;">segmentation variable contain RLE that's is Run Length Enconding. Acc to wikipedia "Run-length encoding (RLE) is a form of lossless data compression in which runs of data (sequences in which the same data value occurs in many consecutive data elements) are stored as a single data value and count, rather than as the original run" like 11WBBBBB will be written as 12W1B5 (that means 1 2times W 1time B 5times</p>

<p style = "color:purple; background-color:pink;">Segmentation variable have NaN value and we will drop that because that is of no use. To know more please go through this <a href = "https://www.youtube.com/watch?v=h6s61a_pqfM" style = "color:green;">RLE</a></p>
</div>

In [ ]:
train_csv.head()

In [ ]:
#get the seperate large , small and stomach rle
train_csv = pd.pivot_table(train_csv, values='segmentation', index='id',columns='class',aggfunc=np.max).astype(str).fillna('')
train_csv = train_csv.reset_index()
train_csv.tail(10)


<p style = "color:purple; background-color:pink;"> id is written like inside folder case123 day 20 folder and then image name is slice_0065 and Note that the image filenames include 4 numbers (ex. 276_276_1.63_1.63.png). These four numbers are slice height / width (integers in pixels) and heigh/width pixel spacing (floating points in mm). The first two defines the resolution of the slide. The last two record the physical size of each pixel.</p>

In [ ]:
x = glob.glob('../input/uw-madison-gi-tract-image-segmentation/train/*/*/*/*')

In [ ]:
#storing image path as the id and all the corresponding info
def setup_img_info(x):
    name = x.split('/')[-1].split('_')
    img_name = ""
    img_name += x.split('/')[-3] + '_' + name[0]+'_'+name[1]
    height,width,h_pixel,w_pixel = (name[2],name[3],name[4],name[5].replace('.png',''))
    #print(img_name , height,width,h_pixel,w_pixel)
    return pd.Series([img_name , height ,width ,h_pixel ,w_pixel])
image_detail = pd.DataFrame()
image_detail['img_path'] = x
image_detail[['id','height','width','h_pixel','w_pixel']] = image_detail['img_path'].apply(lambda t : setup_img_info(t))
    
    
    
    

In [ ]:
image_detail.info()

In [ ]:
df2 = pd.merge(image_detail , train_csv , on='id')

In [ ]:
df2.info()

In [ ]:
df2 = df2.rename(columns = {'large_bowel' : 'rle_large' , 'small_bowel':'rle_small' ,'stomach':'rle_stomach'})

In [ ]:
#df2['rle_large'] = df2['rle_large'].astype(int)
#df2['rle_small'] = df2['rle_small'].astype(int)
#df2['rle_stomach'] = df2['rle_stomach'].astype(int)

In [ ]:
df2.info()

In [ ]:
df2

<span style="color: green; font-family: Segoe UI; font-size: 2.5em; font-weight: 300;">Visualization : Step 2</span>


In [ ]:
import matplotlib.pyplot as plt # linear algebra

def rleToMask(rleString,height,width):
    rows,cols = height,width
    if rleString == 'nan':
        return np.zeros(rows * cols, dtype=np.uint8).reshape(rows,cols)
    else:
        rleNumbers = [int(numstring) for numstring in rleString.split(' ')]
        rlePairs = np.array(rleNumbers).reshape(-1,2)
        img = np.zeros(rows*cols,dtype=np.uint8)
        for index,length in rlePairs:
            index -= 1
            img[index:index+length] = 255
        img = img.reshape(cols,rows)
        img = img.T
        return img

In [ ]:
def visual(row,index):
    plt.figure(figsize=(20,20))
    image = cv2.imread(row['img_path'].iloc[index], cv2.IMREAD_UNCHANGED)
    plt.subplot(1,4,1)
    plt.imshow(image)
    plt.subplot(1,4,2)
    plt.imshow(rleToMask(row['rle_large'].iloc[index],int(row['height'].iloc[index]),int(row['width'].iloc[index])))
    plt.subplot(1,4,3)
    plt.imshow(rleToMask(row['rle_small'].iloc[index],int(row['height'].iloc[index]),int(row['width'].iloc[index])))
    plt.subplot(1,4,4)
    plt.imshow(rleToMask(row['rle_stomach'].iloc[index],int(row['height'].iloc[index]),int(row['width'].iloc[index])))
    #print(index) 
    #break

In [ ]:
for i in range(0,14,3):
    visual(df2,i)